#Inference file for GreenHawak model

This file contains the workflow for the inerference to reproduce the submission using our state of the aart finetuned YOLO8 model for visual pollution detection or infer on new subsets

In [83]:
import os
HOME = os.getcwd()
print(HOME)

/content


In [85]:
# !pip install ultralytics
!git clone https://github.com/asmayamani/GreenHawk.git


Cloning into 'GreenHawk'...
remote: Enumerating objects: 33, done.
remote: Counting objects: 100% (33/33), done.
remote: Compressing objects: 100% (31/31), done.
remote: Total 33 (delta 10), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (33/33), 40.11 MiB | 7.13 MiB/s, done.


In [ ]:
from ultralytics import YOLO
import glob
import pandas as pd
from IPython.display import display, Image
from pathlib import Path    

In [ ]:
!pip install omegaconf

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 KB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 KB 13.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for antlr4-python3-runtime: filename=antlr4_python3_runtime-4.9.3-py3-none-any.whl size=144575 sha256=65d71e4c525052456ff8fbcc893ab6d306bd01076acbcb7056c606b3a14a76e4
  Stored in directory: /root/.cache/pip/wheels/b1/a3/c2/6df046c09459b73cc9bb6c4401b0be6c47048baf9a1617c485
Successfully built antlr4-python3-runtime


In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount= True)


Mounted at /content/drive


#Make the detection with submitted model

In [ ]:
import yaml
config = {
         'train': '/content/drive/MyDrive/Smartathon/TrainF/images/',
         'val': '/content/drive/MyDrive/Smartathon/ValF/images/',
         'nc': 11,
         'names': ['graffiti','faded_signage','potholes','garbage',
                   'construction_road','broken_signage',
                   'bad_streetlight','bad_billboard','sand_on_road','clutter_sidewalk','unkept_facade']}

with open("data.yaml", "w") as file:
    yaml.dump(config, file, default_flow_style=False)

In [86]:
!unzip /content/drive/MyDrive/Smartathon/WRP/Iteration_18/Iteration18a_W.zip


Archive:  /content/drive/MyDrive/Smartathon/WRP/Iteration_18/Iteration18a_W.zip
replace content/runs/detect/train3/weights/best.pt? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: content/runs/detect/train3/weights/best.pt  


In [88]:
#make sure this is the correct path for the test images
test_data = '/content/drive/MyDrive/GreenHawk/Dataset/TestF/images'
weights_path = '/content/GreenHawk/best.pt'


In [ ]:
%cd {HOME}

!yolo task=detect mode=predict source={test_data} model={weights_path} imgsz=640 plots=False  save_txt=True save_conf=True save_crop=False  iou=0.25 conf=0.25

/content
Ultralytics YOLOv8.0.13 🚀 Python-3.8.10 torch-1.13.1+cu116 CPU
Fusing... 
Model summary: 168 layers, 11129841 parameters, 0 gradients, 28.5 GFLOPs
image 1/2092 /content/drive/MyDrive/GreenHawk/Dataset/TestF/images/006e04a34f54d116f6aff2bb31cb320c.jpg: 384x640 1 garbage, 504.6ms
image 2/2092 /content/drive/MyDrive/GreenHawk/Dataset/TestF/images/006e70d6a51af3e670bbe712a83aa1c5.jpg: 384x640 2 garbages, 326.7ms
image 3/2092 /content/drive/MyDrive/GreenHawk/Dataset/TestF/images/0081b25823f23a536c9f1952491db636.jpg: 384x640 2 garbages, 326.5ms
image 4/2092 /content/drive/MyDrive/GreenHawk/Dataset/TestF/images/00b48429c1382cc7d1c72926acedbaa7.jpg: 384x640 2 construction_roads, 1 bad_billboard, 303.5ms
image 5/2092 /content/drive/MyDrive/GreenHawk/Dataset/TestF/images/00b5fc2d3153637e65a35363295e14ba.jpg: 384x640 1 bad_billboard, 320.8ms
image 6/2092 /content/drive/MyDrive/GreenHawk/Dataset/TestF/images/00bce25b1d3a3422bb366a12e3e9ac6a.jpg: 384x640 2 garbages, 1 construction_road, 31

#Prepare the submission files

In [ ]:
names = {'0': 'GRAFFITI', '1': 'FADED_SIGNAGE', '2' : 'POTHOLES', '3':'GARBAGE', '4': 'CONSTRUCTION_ROAD', '5': 'BROKEN_SIGNAGE','6':'' ,'7': 'BAD_BILLBOARD', '8': 'SAND_ON_ROAD', '9' : 'CLUTTER_SIDEWALK' , 
                   '6':'BAD_STREETLIGHT','10':'UNKEPT_FACADE'}

In [ ]:
def yolo_to_xml_bbox(bbox, w, h):
    # x_center, y_center width heigth
    w_half_len = (bbox[2] * w) / 2
    h_half_len = (bbox[3] * h) / 2
    xmin = int((bbox[0] * w) - w_half_len)
    ymin = int((bbox[1] * h) - h_half_len)
    xmax = int((bbox[0] * w) + w_half_len)
    ymax = int((bbox[1] * h) + h_half_len)
    #devide by 2 to match competition expectation
    return [xmax/2,	xmin/2,	ymax/2,	ymin/2]


In [79]:
#Make sure this is the correct path for the predictions made
inferenceFiles = glob.glob('/content/runs/detect/predict/labels/*.txt')

In [80]:
#convert yolo laabels to xml annotation
entries = []
for infPath in inferenceFiles:
  try:
    with open(infPath, 'r') as file2:  # read annotation.txt
        for row in [x.split(' ') for x in file2.read().strip().splitlines()]:
          xml = yolo_to_xml_bbox([float(row[1]),float(row[2]),float(row[3]),float(row[4])],1920,1080)
          entry = [float(row[0]),Path(infPath).name[:-4]+'.jpg',names[row[0]],xml[0],xml[1],xml[2],xml[3]]
          entries.append(entry)

    file2.close()
  except (IOError, OSError) as e:
      print(e)



In [81]:
#add 1px obj
detected = [Path(pth).name[:-4] for pth in inferenceFiles]
print(len(detected))
alltest = [Path(pth).name[:-4] for pth in (glob.glob('/content/drive/MyDrive/Smartathon/TestF/images'+'/*.jpg'))]
undetected = list(set(alltest) - set(detected))

for ftest in undetected:
    entry = [float(6),ftest+'.jpg',names['6'],1,0,1,0]
    entries.append(entry)



1841


In [82]:
#saves the submission
solutionDataFrame = pd.DataFrame(entries,columns= ['class','image_path','name','xmax','xmin','ymax','ymin'])
solutionDataFrame.to_csv('submission18rbest_025_025_s.csv',index=False)